# Lab 3: Policy Search
Solution written by Serra Matteo and Matteo Magnaldi.

In [2]:
import logging
#from minmax import *
from functools import reduce
from math import inf
from copy import deepcopy
from functools import cache
import random

In [22]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self.safe = self.evaluate()
        self.winner = 0  
    
    
    def niming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.info("Yeuch!")

    def evaluate(self) -> bool:
        safe = reduce((lambda x,y: x ^ y), self._rows)
        self._safe = safe == 0
        return self._safe
    
    def is_finished(self) -> bool:
        return all(map(lambda x: x == 0, self._rows))
    
    def __hash__(self):
        return tuple(self._rows).__hash__()


def random_move(nim:Nim):
    rand_row = random.randint(0,len(nim._rows)-1)
    while nim._rows[rand_row] == 0:
        rand_row = random.randint(0,len(nim._rows)-1)

    if nim._rows[rand_row] > nim._k:
        rand_numObj = random.randint(1,nim._k)
    else:
        rand_numObj = random.randint(1,nim._rows[rand_row])

    return rand_row,rand_numObj

def nonZeroRows(nim:Nim):
    return len(nim._rows) - [n == 0 for n in nim._rows].count(True)

def minSum(rows):
    return reduce((lambda x,y: x ^ y), rows)

def expert_system(nim: Nim) -> (int, int):
    # when a single row counts more than an object, invert the strategy 
    invert_strategy = [n > 1 for n in nim._rows].count(True) == 1 and max(nim._rows) <= nim._k
    modRows = [row % (nim._k+1) for row in nim._rows ]
    safe= minSum(modRows) == 0
    if all([n == 1 or n == 0 for n in nim._rows]):
        return random_move(nim)
    else:
        if invert_strategy:
            #when the number of rows whith one remain element will be odd, remove n elements to have all rows with one
            maxElem = max(nim._rows)  
            if nonZeroRows(nim) % 2 == 0 :
                return nim._rows.index(maxElem),maxElem-1
            else:
                return nim._rows.index(maxElem),maxElem
        else:
            if safe:
                # random move or something more clever
                #return random_move(nim)
                #cerca una mossa per cui l'avversario non possa mandarti in sicuro
                for i,n in enumerate(nim._rows):
                    for k in range(1,nim._k+1):
                        if n<k:
                            continue
                        temp_nim = nim._rows[:]
                        temp_nim[i] -= k
                        if minCost(temp_nim)[1] > nim._k:
                            #print("trovata")
                            return i,k
                return random_move(nim)
            else:
                # comeback to safe
                return minCost(modRows)

def random_player() -> (int, int):
    pass

def randomBanchmark(n_iter,n,k,p1_strategy,p2_strategy):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += play(nim,p1_strategy,p2_strategy)
    print(f"winner ratio: {v/n_iter*100}") 
    return 1 if v/n_iter*100 >= 50.00 else 0

def play(nim: Nim,p1_strategy,p2_strategy):
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        if turn:
            row, k = p2_strategy(nim)
        else:
            row,k = p1_strategy(nim)
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)



def possible_actions(nim: Nim):
    
    for i,r in enumerate(nim._rows):
        for e in range(1,nim._k + 1):
            if r >= e:
                
                ret = deepcopy(nim)
                ret.niming(i, e)
                print(f"{ret._rows}")
                yield ret
            else:
                break

# primo int: riga da cui rimuovere, secondo int: quanti elementi rimuovere
@cache
def minmax(state, isMaximizingPlayer, alpha, beta, max_depth = 0):
    current_depth = 0
    if state.is_finished():
        state.winner = 1 if isMaximizingPlayer else -1
        return state.winner
    #print(f"evaluating state: {state._rows}")
    if isMaximizingPlayer:
        best = -inf
        for child in possible_actions(state):
            value = minmax(child, False, alpha, beta, current_depth+1)
            best = max(best, value)
            alpha = max(alpha, best)
            if beta <= alpha:
                break
        return best
    else:
        best = +inf
        for child in possible_actions(state):
            value = minmax(child, True, alpha, beta, current_depth+1)
            best = min(best, value)
            beta = min(beta, best)

            if beta <= alpha:
                break
        return best

def minCost(arr):
     
    cost = inf
    elem = -1
    # calculate XOR sum of array
    XOR = 0
    for e in arr:
         XOR ^= e
 
    # find the min cost and element
    # corresponding
    r_cost = 0
    for i,e in enumerate(arr):
        new_cost = abs((XOR ^ e) - e)
        if (cost > new_cost):
            r_cost = ((XOR ^ e) - e)
            if ((XOR ^ e) - e) > 0: 
                continue
            else:
                cost = (XOR ^ e) - e
            element = e
 
    #print("Element = ", element)
    #print("Operation required = ", r_cost)
    return arr.index(element),-r_cost



In [25]:
#logging.basicConfig(format="%(message)s", level=logging.INFO)
#nim = Nim(4, 3)
#minmax(nim, True, -inf, inf, max_depth= 3)
# for n in possible_actions(nim):
#     print(f"{n._rows}")

randomBanchmark(10000,8,6,expert_system,random_move)
randomBanchmark(10000,8,6,random_move,random_move)
randomBanchmark(10000,8,6,expert_system,expert_system)


winner ratio: 100.0
winner ratio: 50.21
winner ratio: 50.54


In [237]:

NUM_GENERATIONS = 200
POPULATION_SIZE = 20
NIM_DIM = 6
K = 4
NUM_ITER = 100
NUM_ROUND = 200
NUM_COOK_STATUS = 5

def cook_status(nim:Nim):
    cooked = []
    nim_without_zero = [e for e in nim._rows if e !=0 ]
    #longest_row_max_take
    cooked.append((nim._rows.index(max(nim._rows)), nim._k if nim._k< max(nim._rows) else max(nim._rows)))
    #longest_row_min_take
    cooked.append((nim._rows.index(max(nim._rows)), 1))
    #shortest_row_max_take
    cooked.append((nim._rows.index(min(nim_without_zero)), nim._k if nim._k< min(nim_without_zero) else min(nim_without_zero)))
    #shortest_row_min_take
    cooked.append((nim._rows.index(min(nim_without_zero)), 1))
    #random_move
    cooked.append(random_move(nim))

    return cooked

def make_stategy(nim:Nim,probabilities):
    data = cook_status(nim)
    # if round(sum(probabilities),0)!= 1.0:
    #     print("Probabilità errate")
    #     return None
    random_number = random.random()
    prec_probability = 0.0
    for i,e in enumerate(probabilities):
        if random_number < prec_probability + e :
            #print(f"strategy selected {i}")
            return data[i]
        prec_probability += e
    return data[-1]

def playWithProb(nim: Nim,p1_prob,p2_prob):
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        if turn:
            row, k = make_stategy(nim,p2_prob)
        else:
            row,k = make_stategy(nim,p1_prob)
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)

def randomBanchmarkWithProb(n_iter,n,k,p1_prob,p2_prob):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playWithProb(nim,p1_prob,p2_prob)
    #print(f"winner ratio: {v/n_iter*100}")
    return 1 if v/n_iter*100 >= 50.00 else 0

def generateRandomProbability():
    prob = []
    max_range = 1.0
    for i in range(NUM_COOK_STATUS):
        rand_num = random.uniform(0,max_range)
        max_range -= rand_num
        prob.append(rand_num)
    return prob

def tournament(population,tournamen_size = 2):
    return population.index(max(random.choices(population,k=tournamen_size), key=lambda i: i[1]))

def evolution():
    population = [(generateRandomProbability(),0) for _ in range(POPULATION_SIZE)]
    for i in range(NUM_GENERATIONS):
        for j in range(NUM_ROUND):
            index_p1 = tournament(population)
            index_p2 = tournament(population)
            #print(f"p1:{p1[0]},p2:{p2[0]}")
            if randomBanchmarkWithProb(NUM_ITER,NIM_DIM,K,population[index_p1][0],population[index_p2][0]) :
                population[index_p1] = (population[index_p1][0],population[index_p1][1] + 1)
            else: 
                population[index_p2] = (population[index_p2][0],population[index_p2][1] + 1)
            # print(population[index_p1])
            # print(population[index_p1])
        #print(f"population:{population}")
        population = sorted(population,key=lambda i: i[1],reverse=True)[:POPULATION_SIZE//2]
        population = [(e[0],0) for _,e in enumerate(population)]
        population += [(generateRandomProbability(),0) for _ in range(POPULATION_SIZE//2)]
    print(f"best strategy {population[0]}")
    return population[0][0]


In [238]:
best_strategy = evolution()

best strategy ([0.14348417996846008, 0.09246791633851625, 0.7102284378422241, 0.03128760007619407, 0.02013456007470063], 0)


In [239]:
def randomBanchmarkWithProb(n_iter,n,k,p1_prob,p2_prob):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playWithProb(nim,p1_prob,p2_prob)
    print(f"winner ratio: {v/n_iter*100}")
    #return 1 if v/n_iter*100 >= 50.00 else 0
randomBanchmarkWithProb(10000,8,6,best_strategy,[0.0,0.0,0.0,0.0,1.0])
randomBanchmarkWithProb(10000,8,6,best_strategy,[1.0,0.0,0.0,0.0,0.0])
randomBanchmarkWithProb(10000,8,6,best_strategy,[0.0,1.0,0.0,0.0,0.0])
randomBanchmarkWithProb(10000,8,6,best_strategy,[0.0,0.0,1.0,0.0,0.0])
randomBanchmarkWithProb(10000,8,6,best_strategy,[0.0,0.0,0.0,1.0,0.0])

winner ratio: 70.85000000000001
winner ratio: 46.82
winner ratio: 83.82
winner ratio: 46.01
winner ratio: 86.22999999999999


In [252]:
Q = 0.1
NUM_ROUND = 100
def select_stategy(probabilities):
    random_number = random.random()
    prec_probability = 0.0
    for i,e in enumerate(probabilities):
        if random_number < prec_probability + e :
            #print(f"strategy selected {i}")
            return i
        prec_probability += e
    return len(NUM_COOK_STATUS) -1

def playWithStrategy(nim: Nim,p1_strategy,p2_strategy):
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if turn else 'random'}")
    while not nim.is_finished():
        data = cook_status(nim)
        if turn:
            row, k = data[p1_strategy]
        else:
            row,k = data[p2_strategy]
        #print(f"move of {'player'if turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if turn else 'random'}")
    return int(turn)

def randomBanchmarkWithStrategy(n_iter,n,k,p1_strategy,p2_strategy):
    v = 0
    for i in range(n_iter):
        nim = Nim(n,k)
        v += playWithStrategy(nim,p1_strategy,p2_strategy)
    #print(f"winner ratio: {v/n_iter*100}")
    return 1 if v/n_iter*100 >= 50.00 else 0
    

def evolution():
    probabilities = [1/NUM_COOK_STATUS for _ in range(NUM_COOK_STATUS)]
    #for _ in range(NUM_GENERATIONS):
    for _ in range(NUM_ROUND):
            p1_strategy = select_stategy(probabilities)
            p2_strategy = select_stategy(probabilities)
            #print(f"p1:{p1[0]},p2:{p2[0]}")
            if randomBanchmarkWithStrategy(NUM_ITER,NIM_DIM,K,p1_strategy,p2_strategy):
                probabilities[p1_strategy] += Q
                probabilities[p2_strategy] -= Q
            else: 
                probabilities[p2_strategy] += Q
                probabilities[p1_strategy] -= Q
            # print(population[index_p1])
            # print(population[index_p1])
        #print(f"population:{population}")
    print(f"best strategy {probabilities}")
    return probabilities

In [254]:
evolution()

best strategy [0.0, 0.8999999999999999, 0.0, 0.10000000000000003, 2.7755575615628914e-17]


[0.0, 0.8999999999999999, 0.0, 0.10000000000000003, 2.7755575615628914e-17]